
# <p style="text-align: center;">EE 380L: Data Mining</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 40</p>
## <p style="text-align: center;">Due: Thursday, April 24th, submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.  Homework groups will be created and managed through Canvas, so please do not arbitrarily change your homework group.  

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

## Name: 
## UT EID:

## Name:
## UT EID:

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). We will use the Appliances energy prediction dataset for this problem https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction. Use random_state = 42 for the algorithms.

1. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Find the best parameters (*n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (5pts)

2. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR). Find the best parameters (including *n_estimators,* *max_depth* and* learning_rate*), and report corresponding RMSE for each algorithm. (5pts)

3. Use [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Find the best parameters (*n_estimators*, *learning_rate*). Report the accuracy of your model in terms of RMSE. (5pts)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

In [4]:
data = pd.read_csv('energydata_complete.csv') 

y = data['Appliances']
X = data.drop(['date','Appliances', 'rv1', 'rv2'], axis = 1)


from dateutil import parser
timeData = np.array(data['date'])

days = []
hours = []
for line in range(len(timeData)):
    day = parser.parse(timeData[line]).weekday()
    hour = parser.parse(timeData[line]).hour
    days.append(day)
    hours.append(hour)
    
X = pd.concat([X, pd.get_dummies(days), pd.get_dummies(hours)], axis = 1)


x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state=42)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(13222, 56) (13222,) (6513, 56) (6513,)


# Solution

### 1)

In [18]:
param_grid = {"n_estimators": [i for i in range(20, 22)]}
RF_model = RandomForestRegressor(random_state=42)
RF_Grid = GridSearchCV(RF_model, param_grid)
RF_Grid.fit(x_train, y_train)
print(RF_Grid.best_params_)

{'n_estimators': 21}


# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). As in HW4, you will be classifying signal vs background in the MAGIC Gamma Telescope Data Set. The data has been split into training and test as well as standardized for you.

1. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) for the classification. Set the random_state to 42. Find the best parameters (by varying *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

2. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

3. Point out one advantage and one disadvantage of Random Forest compared to GBDT (3pts)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn import preprocessing

In [2]:
# load data
# train
train = pd.read_csv('magic_train.csv', header=None)
y_train = train.values[:,10]
y_train[y_train == 'g'] = 0
y_train[y_train == 'h'] = 1
y_train = y_train.astype(float)
x_train = train.values[:,:10]

# test
test = pd.read_csv('magic_test.csv', header=None)
y_test = test.values[:,10]
y_test[y_test == 'g'] = 0
y_test[y_test == 'h'] = 1
y_test = y_test.astype(float)
x_test = test.values[:,:10]

# standardize the data
x_train = preprocessing.scale(x_train)
x_test = preprocessing.scale(x_test)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


## Solution

### 1）

In [5]:
param_grid = {"n_estimators": [10, 50, 100, 150, 200, 250], "criterion": ["gini", "entropy"]}
RF_model = RandomForestClassifier(random_state=42)
RF_Grid = GridSearchCV(RF_model, param_grid)
RF_Grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100, 150, 200, 250], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
print("Best n_estimators is: " + str(RF_Grid.best_params_["n_estimators"]))
print("Best criterion is: " + str(RF_Grid.best_params_["criterion"]))

Best n_estimators is: 150
Best criterion is: gini


In [22]:
y_predict = RF_Grid.predict(x_test)
a_score = accuracy_score(y_test, y_predict)

roc_score = roc_auc_score(y_test, RF_Grid.predict_proba(x_test)[:,1])

print("Test accuracy (accuracy_score) is: " + str(a_score))
print("Roc_auc_score is: " + str(roc_score))

Test accuracy (accuracy_score) is: 0.8766573621772505
Roc_auc_score is: 0.9326057266732186


### 2)

In [23]:
param_grid = {"n_estimators": [10, 50, 100, 150, 200, 250], "max_depth": [1, 2, 3, 4, 5], "learning_rate": [0.01, 0.05, 0.1, 0.5, 1]}
GBDT_model = GradientBoostingClassifier(random_state=42)
GBDT_Grid = GridSearchCV(GBDT_model, param_grid)
GBDT_Grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100, 150, 200, 250], 'learning_rate': [0.01, 0.05, 0.1, 0.5, 1], 'max_depth': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
print("Best n_estimators for GBDT is: " + str(GBDT_Grid.best_params_["n_estimators"]))
print("Best learning_rate for GBDT is: " + str(GBDT_Grid.best_params_["learning_rate"]))
print("Best max_depth for GBDT is: " + str(GBDT_Grid.best_params_["max_depth"]))

Best n_estimators for GBDT is: 250
Best learning_rate for GBDT is: 0.1
Best max_depth for GBDT is: 5


In [28]:
y_predict = GBDT_Grid.predict(x_test)
a_score = accuracy_score(y_test, y_predict)

roc_score = roc_auc_score(y_test, GBDT_Grid.predict_proba(x_test)[:,1])

print("Test accuracy (accuracy_score) for GBDT is: " + str(a_score))
print("Roc_auc_score for GBDT is: " + str(roc_score))

Test accuracy (accuracy_score) for GBDT is: 0.875785066294487
Roc_auc_score for GBDT is: 0.9324047747088983


In [26]:
param_grid = {"n_estimators": [10, 50, 100, 150, 200, 250], "learning_rate": [0.01, 0.05, 0.1, 0.5, 1]}
Ada_model = AdaBoostClassifier(random_state=42)
Ada_Grid = GridSearchCV(Ada_model, param_grid)
Ada_Grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100, 150, 200, 250], 'learning_rate': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
print("Best n_estimators for AdaBoost is: " + str(Ada_Grid.best_params_["n_estimators"]))
print("Best learning_rate for AdaBoost is: " + str(Ada_Grid.best_params_["learning_rate"]))

Best n_estimators for AdaBoost is: 200
Best learning_rate for AdaBoost is: 1


In [30]:
y_predict = Ada_Grid.predict(x_test)
a_score = accuracy_score(y_test, y_predict)

roc_score = roc_auc_score(y_test, Ada_Grid.predict_proba(x_test)[:,1])

print("Test accuracy (accuracy_score) for AdaBoost is: " + str(a_score))
print("Roc_auc_score for AdaBoost is: " + str(roc_score))

Test accuracy (accuracy_score) for AdaBoost is: 0.8473482205163991
Roc_auc_score for AdaBoost is: 0.9000435473322483


### 3)

One advantage of Random Forest compared to GBDT is: Random Forest is easier to parallelize while GBDT is not so trivial to achieve it. For each tree of Random Forest, we use bootstrap method to choose smaples. We also choose a random subset of the festures to build the model. In this case, the training process of different trees will not interact with each other, and thus it is easier to be splitted and computed synchronously in multiple threads. By the way, Random Forest is usually easier to tune.

One disadvantage of Random Forest compared to GBDT is: GBDT performs better on weak learners with high bias and low variance. Each step, a new model is built to compenstae the already built model. So if our model is of relatively low accuracy, GBDT can be a better choice.

# Question 3: Visualization using Bokeh (10 pts)

In this problem, you'll build an interactive visualization. Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. For more information on Bokeh, see http://bokeh.pydata.org/en/latest/. The problem statement is as follows:

Using the "nbasalaries_new.csv" data set, your goal is to build a Bokeh visualization which allows the user to explore how salary (on a log scale) varies with points per game (PSG) and age. You will create a visualization that allows the user to toggle the X axis of a scatter plot between PSG and age, with the y-axis always being log Salary. Also add the hover tool so that if the user hovers over a datapoint in the plot a window pops up that shows the player name, team, position, salary, and the current x variable (PSG or age) depending on the current tab.  Add the ability to Zoom in/out.

Hints: 
1. see: http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#basic-tooltips for hover and zoom tool examples.
2. See: http://bokeh.pydata.org/en/latest/docs/reference/plotting.html. Look for the scatter API.
3. See: http://bokeh.pydata.org/en/0.10.0/docs/user_guide/styling.html#labels. For labeling axes.
4. See: https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html  for how to use jitter transform
5. Use output_file() from Bokeh to output the plot to an external file. Submit this file as well.

In [1]:
import pandas as pd
import numpy as np
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, BoxZoomTool, Jitter
from bokeh.plotting import figure, output_notebook, show, output_file

data = pd.read_csv("nbasalaries_new.csv")
data["logsalary"] = data.SALARY.apply(np.log)

## Solution